In [1]:
AminoAcidMass = {
    'G': 57, 'A': 71, 'S': 87, 'P': 97, 'V': 99, 'T': 101,
    'C': 103, 'I': 113, 'L': 113, 'N': 114, 'D': 115, 'K': 128,
    'Q': 128, 'E': 129, 'M': 131, 'H': 137, 'F': 147, 'R': 156,
    'Y': 163, 'W': 186}

O algoritmo de sequenciamento de ciclopeptídeos de força bruta 'BFCyclopeptideSequencing' gera todos os peptídeos possíveis cuja massa é igual a 'ParentMass(Spectrum)' e então verifica qual desses peptídeos tem espectros teóricos correspondentes ao Spectrum .


```
BFCyclopeptideSequencing(Spectrum)
    for every peptide with Mass(Peptide) equal to ParentMass(Spectrum)
        if Spectrum = Cyclospectrum(Peptide)
            output Peptide
```

In [2]:
def BFCyclopeptideSequencing(Spectrum, AminoAcidMass):
    """
    Implementa o algoritmo de Força Bruta (Branch-and-Bound) para resolver o problema de sequenciamento de peptídeos cíclicos.

    Parâmetros:
        Spectrum: lista de inteiros representando o espectro de massas (Spectrum experimental).
        AminoAcidMass: dicionário onde as chaves são os aminoácidos e os valores são suas respectivas massas.

    Saída:
        Lista de peptídeos cujos espectros cíclicos correspondem ao Spectrum fornecido.
    """
    def ParentMass(Spectrum):
        return max(Spectrum)  # A maior massa no espectro

    def Mass(Peptide):
        return sum(AminoAcidMass[aa] for aa in Peptide)  # Massa total de um peptídeo

    def Cyclospectrum(Peptide):
        # Gera o espectro cíclico de um peptídeo
        n = len(Peptide)
        extended_peptide = Peptide + Peptide  # Para facilitar o "wrap around"
        prefix_mass = [0] * (2 * n + 1)

        for i in range(1, len(extended_peptide) + 1):
            prefix_mass[i] = prefix_mass[i - 1] + AminoAcidMass[extended_peptide[i - 1]]

        spectrum = [0]
        peptide_mass = prefix_mass[n]

        for i in range(n):
            for j in range(i + 1, i + n + 1):
                spectrum.append(prefix_mass[j] - prefix_mass[i])
                if j - i < n:
                    spectrum.append(peptide_mass - (prefix_mass[j] - prefix_mass[i]))

        return sorted(spectrum)

    def Expand(Peptides):
        # Expande cada peptídeo adicionando cada aminoácido possível
        expanded = []
        for peptide in Peptides:
            for aa in AminoAcidMass.keys():
                expanded.append(peptide + aa)
        return expanded

    Peptides = [""]  # Começa com um peptídeo vazio
    FinalPeptides = []  # Lista para armazenar os peptídeos finais
    parent_mass = ParentMass(Spectrum)  # Massa total do espectro

    while Peptides:
        Peptides = Expand(Peptides)  # Expande os peptídeos
        for peptide in Peptides[:]:  # Itera por uma cópia para evitar remoções conflitantes
            if Mass(peptide) == parent_mass:
                if Cyclospectrum(peptide) == Spectrum:
                    FinalPeptides.append(peptide)
                Peptides.remove(peptide)
            elif Mass(peptide) > parent_mass:
                Peptides.remove(peptide)

    return FinalPeptides

In [3]:
def Cyclospectrum_2(Peptide):
    # Gera o espectro cíclico de um peptídeo
    n = len(Peptide)
    extended_peptide = Peptide + Peptide  # Para facilitar o "wrap around"
    prefix_mass = [0] * (2 * n + 1)

    for i in range(1, len(extended_peptide) + 1):
        prefix_mass[i] = prefix_mass[i - 1] + AminoAcidMass[extended_peptide[i - 1]]

    spectrum = [0]
    peptide_mass = prefix_mass[n]

    for i in range(n):
        for j in range(i + 1, i + n + 1):
            spectrum.append(prefix_mass[j] - prefix_mass[i])
            if j - i < n:
                spectrum.append(peptide_mass - (prefix_mass[j] - prefix_mass[i]))

    return sorted(spectrum)

def generate_all_peptides(ParentMass):
    # Geração manual de peptídeos sem itertools
    amino_acids = list(AminoAcidMass.keys())

    # Função auxiliar para calcular a massa de um peptídeo
    def peptide_mass(peptide):
        return sum(AminoAcidMass[aa] for aa in peptide)

    # Gera todos os peptídeos por recursão
    def build_peptides(current_peptide, remaining_mass):
        if remaining_mass == 0:
            peptides.append(current_peptide)
            return
        for aa in amino_acids:
            mass = AminoAcidMass[aa]
            if mass <= remaining_mass:
                build_peptides(current_peptide + aa, remaining_mass - mass)

    peptides = []
    build_peptides("", ParentMass)
    return peptides

def BFCyclopeptideSequencing_2(Spectrum):
    # Determinar ParentMass
    ParentMass = max(Spectrum)

    # Gerar todos os peptídeos possíveis
    possible_peptides = generate_all_peptides(ParentMass)

    # Filtrar os peptídeos válidos
    valid_peptides = []
    for Peptide in possible_peptides:
        if Cyclospectrum_2(Peptide) == Spectrum:
            valid_peptides.append(Peptide)

    return valid_peptides